In [1]:
import sys
sys.path.append("../")

In [2]:
from utils.common_train_utils import *
from utils.draw_candle_image import *
from utils.evaluate_old_models import *
from functools import partial
import os
from tabulate import tabulate
import random
import warnings
warnings.filterwarnings("ignore")


In [3]:
def print_loss_of_all_models_in_dataset_folder(folder_object):
    dataset_path = folder_object.path
    
    config = read_config(dataset_path)
    candle_type_and_directory_save: dict = config["candle_type_and_directory_save"]
    days_result = config["days_result"]
    
    dataset_test = load_dataset_of_each_type_and_combine(f"{dataset_path}/test", candle_type_and_directory_save)
    
    batch_size = random.randint(0, int(len(dataset_test) / 4))
    shuffle_buffer = random.randint(0, int(len(dataset_test) / 3))
    
    dataset_test_1 = dataset_test.shuffle(shuffle_buffer)
    dataset_test_2 = dataset_test_1.batch(batch_size)
    
    # Ví dụ
    all_models_folder_path = os.path.join(os.path.abspath(dataset_path), "model_save")
    df = evaluate_all_models_in_dataset_folder(
        all_models_folder_path, 
        dataset_test_2,
        days_result
    )
    
    # print(f"----------------- {folder_name} -----------------")
    # print(df)
    
    df["total_records"] = len(dataset_test)
    df["batch_size"] = batch_size
    df["shuffle_buffer"] = shuffle_buffer
    
    return df

In [4]:
dataset_parent_folder = os.path.abspath("../dataset")

In [5]:
all_dfs = {}
for f in os.scandir(dataset_parent_folder):
    if not f.is_dir(): continue
    
    df = print_loss_of_all_models_in_dataset_folder(f)
    all_dfs[f.name] = df

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - loss: 4.2806 - mae: 1.3659 - mse: 4.2806
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - loss: 4.2412 - mae: 1.3237 - mse: 4.2412
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - loss: 4.2848 - mae: 1.3209 - mse: 4.2848
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - loss: 4.3271 - mae: 1.3203 - mse: 4.3271
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - loss: 4.1435 - mae: 1.2864 - mse: 4.1435 
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 3.9517 - mae: 1.2740 - mse: 3.9517
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - loss: 4.1417 - mae: 1.3028 - mse: 4.1417
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 3.5440 - mae: 1.3298 - mse: 3.5440
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 2.9710 - mae: 1.1800 - mse: 2.9710
221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 1.5325 - mae: 0.8952 - mse: 1.5325
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 1.3720 - mae: 0.7912 - mse: 1.3720
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 1.3709 - mae: 0.8099 - mse: 1.370

In [6]:
list(all_dfs.keys())

['btc_dataset_0_2_3_2014_2024_test_2023_2024_with_ema_macd_trend',
 'btc_dataset_0_2_3_2014_2024_test_2024_2024_with_ema_macd_trend',
 'coca_cola_dataset_0_2_3_1980_2024_test_2017_2024_with_ema_macd_trend',
 'ctg_dataset_0_2_3_2009_2024_test_2023_2024_with_ema_macd_trend',
 'fpt_dataset_0_2_3_2012_2024_test_2023_2024_with_ema_macd_trend',
 'tcb_dataset_0_2_3_2018_2024_test_2024_2024_with_ema_macd_trend',
 'vn30_dataset_0_2_3_2012_2024_test_2023_2024_with_ema_macd_trend']

In [7]:
for dataset_name in all_dfs:
    print(dataset_name)
    print(tabulate(all_dfs[dataset_name], headers='keys', tablefmt='pretty'))
    print("\n")

btc_dataset_0_2_3_2014_2024_test_2023_2024_with_ema_macd_trend
+---+-----------------------------------------------+-------------------+--------------------+---------------+------------+----------------+
|   |                  model_name                   |        mse        |        mae         | total_records | batch_size | shuffle_buffer |
+---+-----------------------------------------------+-------------------+--------------------+---------------+------------+----------------+
| 5 |  model_5_biLSTM_ema_macd_trend_cl_test_1_1_1  | 4.002681255340576 | 1.284140944480896  |      364      |     24     |       66       |
| 4 |   model_5_biLSTM_ema_macd_trend_cl_test_1_1   | 4.007717132568359 | 1.2823755741119385 |      364      |     24     |       66       |
| 6 | model_5_biLSTM_ema_macd_trend_cl_test_1_1_1_1 | 4.017830848693848 |  1.27968168258667  |      364      |     24     |       66       |
| 3 |    model_5_biLSTM_ema_macd_trend_cl_test_1    | 4.031723976135254 | 1.280347108840942

In [8]:
btc_dataset_0_2_3_2014_2024_test_2024_2024_with_ema_macd_trend
+---+---------------------------------------------+-------------------+--------------------+---------------+------------+----------------+
|   |                 model_name                  |        mse        |        mae         | total_records | batch_size | shuffle_buffer |
+---+---------------------------------------------+-------------------+--------------------+---------------+------------+----------------+
| 6 |  model_5_biLSTM_ema_macd_trend_1_1_1_1_1_1  | 4.804694652557373 | 1.429203748703003  |      57       |     14     |       15       |
| 7 | model_5_biLSTM_ema_macd_trend_1_1_1_1_1_1_1 | 4.804695129394531 | 1.4292036294937134 |      57       |     14     |       15       |
| 4 |    model_5_biLSTM_ema_macd_trend_1_1_1_1    | 4.921758651733398 | 1.4760667085647583 |      57       |     14     |       15       |
| 5 |   model_5_biLSTM_ema_macd_trend_1_1_1_1_1   | 4.921758651733398 | 1.4760668277740479 |      57       |     14     |       15       |
| 2 |      model_5_biLSTM_ema_macd_trend_1_1      | 5.724715232849121 | 1.5685756206512451 |      57       |     14     |       15       |
| 3 |     model_5_biLSTM_ema_macd_trend_1_1_1     | 5.724715709686279 | 1.5685756206512451 |      57       |     14     |       15       |
| 0 |        model_5_biLSTM_ema_macd_trend        | 5.784203052520752 | 1.541532278060913  |      57       |     14     |       15       |

SyntaxError: invalid syntax (146829886.py, line 2)